In [28]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
import os

from torchvision import transforms
import torch.optim as optim

import torch.nn.functional as F


from PIL import Image

#from torchmetrics import JaccardIndex
#from thop import profile, clever_format
import time

#from fvcore.nn import FlopCountAnalysis, flop_count_table
#import torchprofile

#from models.deeplabv2.deeplabv2 import get_deeplab_v2


#from torchmetrics import JaccardIndex
#from thop import profile, clever_format


#from fvcore.nn import FlopCountAnalysis, flop_count_table
#import torchprofile





**Cityscapes Dataset Download**

In [29]:
#!pip install gdown



In [30]:
#import gdown

#url = 'https://drive.google.com/uc?id=1Qb4UrNsjvlU-wEsR9d7rckB0YS_LXgb2'
#output = 'your_dataset.zip'  # Change this to the desired output filename
#gdown.download(url, output, quiet=False)
from google.colab import drive

drive.mount('/content/Drive')
get_ipython().system('/content/Drive/MyDrive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
/bin/bash: line 1: /content/Drive/MyDrive: Is a directory


In [31]:
#get_ipython().system('unzip /content/your_dataset.zip -d /content/')

**GTA5 Dataset download**

dataset link :

In [32]:
#url=' https://drive.google.com/uc?id=1xYxlcMR2WFCpayNrW2-Rb7N-950vvl23'
#output = 'GTA5.zip'  # Change this to the desired output filename
#gdown.download(url, output, quiet=False)

In [33]:
#get_ipython().system('unzip /content/GTA5.zip -d /content/')

**Model Clone**

In [34]:
# Clone the GitHub repository
!git clone https://github.com/Gabrysse/MLDL2024_project1.git


fatal: destination path 'MLDL2024_project1' already exists and is not an empty directory.


In [35]:
# Navigate to the project directory
%cd MLDL2024_project1

/content/MLDL2024_project1/MLDL2024_project1


In [36]:
#from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet

In [37]:
import torch

In [38]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


**Parameters**

In [39]:
# Define training parameters
epochs = 50
learning_rate = 0.01
batch_size = 4
train_resolution = (1280,720) #(1024, 512)
test_resolution = (1024, 512)


In [40]:

class CityscapesDataset(Dataset):
  def __init__(self, root_dir, im_transform ):

    """
    Args:
    root_dir (string): Directory with all the images.
    transform (callable, optional): Optional transform to be applied on a sample.
    """
    self.root_dir = root_dir
    self.im_transform = im_transform
    #self.lab_transform = lab_transform
    self.images = []
    for subdir, dirs, files in os.walk(root_dir):
      for file in files:
        if file.endswith('_gtFine_color.png'):
          self.images.append(os.path.join(subdir, file))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_name = self.images[idx]
    image = Image.open(img_name).convert('RGB')
    label_name = img_name.replace('_gtFine_color.png', '_gtFine_labelTrainIds.png')  #labelTrainIds
    label = Image.open(label_name)


    # Resize label using nearest-neighbor interpolation
    label = TF.resize(label, (1024, 512), interpolation=transforms.InterpolationMode.NEAREST)
    label = np.array(label)  # Convert to numpy array
    label = torch.from_numpy(label).long()  # Convert to LongTensor


    if self.im_transform:

      image = self.im_transform(image)

    # if self.lab_transform:
    #   label = self.lab_transform(label)

    return image, label



In [41]:
class GTA5Dataset(Dataset):
  def __init__(self, root_dir, im_transform=None):
    """
    Args:
    root_dir (string): Directory with all the images and labels.
    im_transform (callable, optional): Optional transform to be applied on a sample.
    """
    self.root_dir = root_dir
    self.im_transform = im_transform
    self.images_dir = os.path.join(root_dir, 'images')
    self.labels_dir = os.path.join(root_dir, 'labels')
    self.images = [os.path.join(self.images_dir, file) for file in os.listdir(self.images_dir) if file.endswith('.png')]

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_name = self.images[idx]
    image = Image.open(img_name).convert('RGB')
    label_name = img_name.replace('images', 'labels')  # Assuming label images are in the 'labels' folder with the same name
    label = Image.open(label_name)

    # Resize label using nearest-neighbor interpolation
    label = TF.resize(label, (1280,720) , interpolation=transforms.InterpolationMode.NEAREST)
    label = np.array(label)  # Convert to numpy array

    # Define the maximum valid label ID
    max_valid_id = 18
    # Set all label IDs greater than max_valid_id to the 'ignore' class (e.g., 255)
    label[label > max_valid_id] = 255


    label = torch.from_numpy(label).long()  # Convert to LongTensor

    if self.im_transform:
      image = self.im_transform(image)

    return image, label


**GTA5 Augmented Train Loader**

In [42]:
# Create the dataset

# im_transform=transforms.Compose([

# transforms.ToTensor(),
# transforms.Resize(train_resolution),

# ])


###-----   Augmentation Transforms:

# Create the dataset with augmentations
im_transform = transforms.Compose([
transforms.ToTensor(),
transforms.Resize(train_resolution),

#transforms.RandomHorizontalFlip(p=0.5),  # Apply horizontal flip with 50% probability
transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.5)  # Apply Gaussian Blur with 50% probability
#transforms.RandomApply([transforms.ColorJitter(brightness=0.5)], p=0.5),  # Apply brightness adjustment with 50% probability


])



gta5_dataset = GTA5Dataset(root_dir='/content/Drive/MyDrive/GTA5', im_transform=im_transform)




In [43]:
gta5_loader = DataLoader(gta5_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [44]:
len(gta5_loader.dataset)

2500

**Cityscapes Train Data Loader**

In [45]:
# Define a transform if you need to preprocess the images
transformed_dataset = CityscapesDataset(root_dir='/content/Drive/MyDrive/Cityspaces/Cityspaces/gtFine/train', #/content/Cityscapes/Cityspaces/gtFine/train
im_transform=transforms.Compose([

transforms.ToTensor(),
transforms.Resize(train_resolution),

]), )


cityscape_train_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4)




In [46]:
len(cityscape_train_loader.dataset)

1572

**Cityscapes Validation Data Loader**

In [47]:
# Define a transform if you need to preprocess the images
transformed_dataset = CityscapesDataset(root_dir='/content/Drive/MyDrive/Cityspaces/Cityspaces/gtFine/val', #/content/Cityscapes/Cityspaces/gtFine/train
im_transform=transforms.Compose([

transforms.ToTensor(),
transforms.Resize(test_resolution),

]), )


cityscape_val_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4)




In [48]:
len(cityscape_val_loader.dataset)

500

In [49]:
# Load the DeepLabV2 model
#model = get_deeplab_v2(num_classes=19, pretrain=False)

# Load the BiSeNet model
model = BiSeNet(num_classes=19,context_path='resnet18') #'resnet18')  #'resnet101')  #BiSeNet


model = model.to('cuda')



# Define the CrossEntropyLoss with ignore_index set to 255
criterion = nn.CrossEntropyLoss(ignore_index=255)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


**IoU Calculation Function for Classes**

In [50]:
def fast_hist(a, b, n):
    '''
    a and b are label and prediction respectively
    n is the number of classes
    '''
    #k = (a >= 0) & (a < n)
    k = (b >= 0) & (b < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)


def per_class_iou(hist):
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

**Discriminator Model**

In [51]:
class FCDiscriminator(nn.Module):
  def __init__(self, num_classes, ndf=64):
    super(FCDiscriminator, self).__init__()
    self.conv1 = nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1)
    self.conv2 = nn.Conv2d(ndf, ndf*2, kernel_size=4, stride=2, padding=1)
    self.conv3 = nn.Conv2d(ndf*2, ndf*4, kernel_size=4, stride=2, padding=1)
    self.conv4 = nn.Conv2d(ndf*4, ndf*8, kernel_size=4, stride=2, padding=1)
    self.classifier = nn.Conv2d(ndf*8, 1, kernel_size=4, stride=2, padding=1)
    self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

  def forward(self, x):
    x = self.conv1(x)
    x = self.leaky_relu(x)
    x = self.conv2(x)
    x = self.leaky_relu(x)
    x = self.conv3(x)
    x = self.leaky_relu(x)
    x = self.conv4(x)
    x = self.leaky_relu(x)
    x = self.classifier(x)
    return x


**BiSeNet Model Loading**

In [52]:
# Load the BiSeNet model
model = BiSeNet(num_classes=19,context_path='resnet18') #'resnet18')  #'resnet101')  #BiSeNet


model = model.to(device)


**Adversarial Training**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import itertools

# Assuming 'model' is your segmentation model and 'FCDiscriminator' is defined as above
# Initialize models
model = model.to('cuda')
discriminator = FCDiscriminator(num_classes=19).to('cuda')  # Number of classes for segmentation task

# Optimizers
optimizer_model = optim.Adam(model.parameters(), lr=learning_rate)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Loss functions
criterion_segmentation = nn.CrossEntropyLoss(ignore_index=255)  # Loss function for segmentation model
criterion_discriminator = nn.BCEWithLogitsLoss()                # Loss function for discriminator model

# Create an infinite iterator for the smaller dataset
target_iter = itertools.cycle(cityscape_train_loader)   # target is smaller

# Main training loop
for epoch in range(epochs):
    # Reset the target iterator every epoch, because it is a larger dataset
    source_iter = iter(gta5_loader)

    # If source image loader is the larger dataset
    for _ in range(len(gta5_loader)):
        # Get a batch from the source dataset
        source_data = next(source_iter)
        source_images, source_labels = source_data
        source_images = source_images.to('cuda')
        source_labels = source_labels.to('cuda')

        # Get a batch from the target dataset
        target_data = next(target_iter)
        target_images, _ = target_data
        target_images = target_images.to('cuda')

        ############### Train segmentation model with source and target ##################

        ########## Train segmentation model with source images ##########
        model.train()
        optimizer_model.zero_grad()
        source_pred = model(source_images)
        loss_segmentation = criterion_segmentation(source_pred[0], source_labels)
        loss_segmentation.backward()

        ########## Train segmentation model with target images ##########
        target_pred = model(target_images)
        target_pred_softmax = F.softmax(target_pred[0], dim=1)  # Apply softmax to the model output

        # Don't accumulate grads in Discriminator
        for param in discriminator.parameters():
            param.requires_grad = False

        pred_target = discriminator(target_pred_softmax)  # Use softmax output as input for discriminator

        # Very important: Label as 1 for target to fool discriminator
        target_label = torch.ones_like(pred_target).to('cuda')

        loss_adv = criterion_discriminator(pred_target, target_label)

        # Backpropagate the adversarial loss through the segmentation model only
        loss_adv.backward()  # Backpropagate loss to update only segmentation model's parameters

        ############# Train discriminator with source and target ###########################

        ########## Train discriminator with source ##########
        source_pred = model(source_images)
        source_pred = source_pred[0].detach()  # Detach source_pred from the graph to prevent gradients from flowing into 'model'

        # Enable gradient computation for discriminator parameters
        for param in discriminator.parameters():
            param.requires_grad = True

        discriminator_out = discriminator(F.softmax(source_pred, dim=1))

        # Calculate loss for discriminator
        source_label = torch.ones_like(discriminator_out).to('cuda')  # Label as 1 for source
        loss_adv = criterion_discriminator(discriminator_out, source_label)

        optimizer_discriminator.zero_grad()
        loss_adv.backward()

        ########## Train discriminator with target ##########
        target_pred = model(target_images)
        target_pred = target_pred[0].detach()  # Detach target_pred from the graph to prevent gradients from flowing into 'model'

        discriminator_out = discriminator(F.softmax(target_pred, dim=1))

        # Calculate loss for discriminator
        target_label = torch.zeros_like(discriminator_out).to('cuda')  # Label as 0 for target
        loss_adv = criterion_discriminator(discriminator_out, target_label)
        loss_adv.backward()

        # Update the models
        optimizer_model.step()
        optimizer_discriminator.step()




**Inference and mIoU calculation on test set**

In [ ]:



model.eval()


all_classes_iou=np.zeros(19)
test_counter=0

with torch.no_grad():
    for images, labels in cityscape_val_loader:          #test_loader:
        images, labels = images.to(device), labels.to(device)


        outputs = model(images)

        # Resize predictions to the original Cityscapes resolution
        outputs = TF.resize(outputs, test_resolution, interpolation=Image.NEAREST)




        #calcualting IoU for each class and for each (image,label) pair separately

        for i in range (batch_size):

          #mask=labels[i,:,:].cpu().numpy().flatten() != 255
          #hist=fast_hist(outputs[i,:,:,:].argmax(dim=0).cpu().numpy().flatten()[mask] , labels[i,:,:].cpu().numpy().flatten()[mask], 19)

          hist=fast_hist(outputs[i,:,:,:].argmax(dim=0).cpu().numpy().flatten() , labels[i,:,:].cpu().numpy().flatten(), 19)

          all_classes_iou=all_classes_iou+per_class_iou(hist)
          test_counter = test_counter +1









In [ ]:
test_counter

In [ ]:
all_classes_mIOU=(all_classes_iou/test_counter).round(3)  #calculating mean intersection over union for each class

In [ ]:
all_classes_mIOU

In [ ]:
mIoU=all_classes_mIOU.mean()  #

In [ ]:
mIoU

**GTA5 Labels**

In [ ]:
from abc import ABCMeta
from dataclasses import dataclass
from typing import Tuple


class BaseGTALabels(metaclass=ABCMeta):
    pass


@dataclass
class GTA5Label:
    ID: int
    color: Tuple[int, int, int]


class GTA5Labels_TaskCV2017(BaseGTALabels):
    road = GTA5Label(ID=0, color=(128, 64, 128))
    sidewalk = GTA5Label(ID=1, color=(244, 35, 232))
    building = GTA5Label(ID=2, color=(70, 70, 70))
    wall = GTA5Label(ID=3, color=(102, 102, 156))
    fence = GTA5Label(ID=4, color=(190, 153, 153))
    pole = GTA5Label(ID=5, color=(153, 153, 153))
    light = GTA5Label(ID=6, color=(250, 170, 30))
    sign = GTA5Label(ID=7, color=(220, 220, 0))
    vegetation = GTA5Label(ID=8, color=(107, 142, 35))
    terrain = GTA5Label(ID=9, color=(152, 251, 152))
    sky = GTA5Label(ID=10, color=(70, 130, 180))
    person = GTA5Label(ID=11, color=(220, 20, 60))
    rider = GTA5Label(ID=12, color=(255, 0, 0))
    car = GTA5Label(ID=13, color=(0, 0, 142))
    truck = GTA5Label(ID=14, color=(0, 0, 70))
    bus = GTA5Label(ID=15, color=(0, 60, 100))
    train = GTA5Label(ID=16, color=(0, 80, 100))
    motocycle = GTA5Label(ID=17, color=(0, 0, 230))
    bicycle = GTA5Label(ID=18, color=(119, 11, 32))

    list_ = [
        road,
        sidewalk,
        building,
        wall,
        fence,
        pole,
        light,
        sign,
        vegetation,
        terrain,
        sky,
        person,
        rider,
        car,
        truck,
        bus,
        train,
        motocycle,
        bicycle,
    ]

    @property
    def support_id_list(self):
        ret = [label.ID for label in self.list_]
        return ret

In [ ]:
class_labels = []
for label_name, label in GTA5Labels_TaskCV2017.__dict__.items():
  if isinstance(label, GTA5Label):
    class_labels.append(label_name)

In [ ]:
class_labels

In [ ]:
import pandas as pd

In [ ]:
# Create a DataFrame
all_classes_mIOU_DF = pd.DataFrame({
'class_Label': class_labels,
'class_IoU': all_classes_iou
})

all_classes_mIOU_DF
